In [ ]:
import asyncio
import time
import json

from indy import anoncreds, did, ledger, pool, wallet, IndyError
from indy.error import ErrorCode

async def run():
    print("Getting started -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    print("\n=====================================================================")
    print("=== Getting Endorser credentials for Bank, landlord, Thrift and rental")


    steward = {
        'name': "Sovrin Steward",
        'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
        'wallet_credentials': json.dumps({'key': 'steward_wallet_key'}),
        'pool': pool_['handle'],
        'seed': '000000000000000000000000Steward1'
    }

    await create_wallet(steward)

    print("\"Sovrin Steward\" -> Create and store in Wallet DID from seed")
    steward['did_info'] = json.dumps({'seed': steward['seed']})
    steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])

    print("\n=====================================================================")
    print("== Getting Endorser credentials - RentalCompany getting Verinym")


    rental = {
        'name': 'Rentalcompany',
        'wallet_config': json.dumps({'id': 'rental_wallet'}),
        'wallet_credentials': json.dumps({'key': 'rental_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'ENDORSER'
    }

    await getting_verinym(steward, rental)

    print("\n=====================================================================")
    print("== Getting Endorser credentials - bank getting Verinym")


    bank = {
        'name': 'Bankorganization',
        'wallet_config': json.dumps({'id': 'bank_wallet'}),
        'wallet_credentials': json.dumps({'key': 'bank_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'ENDORSER'
    }

    await getting_verinym(steward, bank)

    print("\n=====================================================================")
    print("== Getting Endorser credentials - Landlord getting Verinym")


    landlord = {
        'name': 'Landlord',
        'wallet_config': json.dumps({'id': 'landlord_wallet'}),
        'wallet_credentials': json.dumps({'key': 'landlord_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'ENDORSER'
    }

    await getting_verinym(steward, landlord)

    print("\n=====================================================================")
    print("== Getting Endorser credentials - Thrift getting Verinym")


    thrift = {
        'name': 'Thrift',
        'wallet_config': json.dumps({'id': 'thrift_wallet'}),
        'wallet_credentials': json.dumps({'key': 'thrift_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'ENDORSER'
    }

    await getting_verinym(steward, thrift)

    print("\n=====================================================================")
    print("=== Credential Schemas Setup ==")


    print("\"rental\" -> Create \"Job-Certificate\" Schema")
    job_certificate = {
        'name': 'Job-Certificate',
        'version': '0.2',
        'attributes': ['first_name', 'last_name', 'salary', 'employee_status', 'experience']
    }
    (rental['job_certificate_schema_id'], rental['job_certificate_schema']) = \
        await anoncreds.issuer_create_schema(rental['did'], job_certificate['name'], job_certificate['version'],
                                             json.dumps(job_certificate['attributes']))
    job_certificate_schema_id = rental['job_certificate_schema_id']

    print("\"rental\" -> Send \"Job-Certificate\" Schema to Ledger")
    await send_schema(rental['pool'], rental['wallet'], rental['did'], rental['job_certificate_schema'])

    print("\"rental\" -> Create \"Transcript\" Schema")
    transcript = {
        'name': 'Transcript',
        'version': '1.2',
        'attributes': ['first_name', 'last_name', 'degree', 'status', 'year', 'average', 'ssn']
    }
    (rental['transcript_schema_id'], rental['transcript_schema']) = \
        await anoncreds.issuer_create_schema(rental['did'], transcript['name'], transcript['version'],
                                             json.dumps(transcript['attributes']))
    transcript_schema_id = rental['transcript_schema_id']

    print("\"rental\" -> Send \"Transcript\" Schema to Ledger")
    await send_schema(rental['pool'], rental['wallet'], rental['did'], rental['transcript_schema'])

    time.sleep(1)  # sleep 1 second before getting schema

    print("\n=====================================================================")
    print("=== bank Credential Definition Setup ==")


    print("\"bank\" -> Get \"Transcript\" Schema from Ledger")
    (bank['transcript_schema_id'], bank['transcript_schema']) = \
        await get_schema(bank['pool'], bank['did'], transcript_schema_id)

    print("\"bank\" -> Create and store in Wallet \"bank Transcript\" Credential Definition")
    transcript_cred_def = {
        'tag': 'TAG1',
        'type': 'CL',
        'config': {"support_revocation": False}
    }
    (bank['transcript_cred_def_id'], bank['transcript_cred_def']) = \
        await anoncreds.issuer_create_and_store_credential_def(bank['wallet'], bank['did'],
                                                               bank['transcript_schema'], transcript_cred_def['tag'],
                                                               transcript_cred_def['type'],
                                                               json.dumps(transcript_cred_def['config']))

    print("\"bank\" -> Send  \"bank Transcript\" Credential Definition to Ledger")
    await send_cred_def(bank['pool'], bank['wallet'], bank['did'], bank['transcript_cred_def'])

    print("\n=====================================================================")
    print("=== landlord Credential Definition Setup ==")


    print("\"landlord\" -> Get from Ledger \"Job-Certificate\" Schema")
    (landlord['job_certificate_schema_id'], landlord['job_certificate_schema']) = \
        await get_schema(landlord['pool'], landlord['did'], job_certificate_schema_id)

    print("\"landlord\" -> Create and store in Wallet \"landlord Job-Certificate\" Credential Definition")
    job_certificate_cred_def = {
        'tag': 'TAG1',
        'type': 'CL',
        'config': {"support_revocation": False}
    }
    (landlord['job_certificate_cred_def_id'], landlord['job_certificate_cred_def']) = \
        await anoncreds.issuer_create_and_store_credential_def(landlord['wallet'], landlord['did'],
                                                               landlord['job_certificate_schema'],
                                                               job_certificate_cred_def['tag'],
                                                               job_certificate_cred_def['type'],
                                                               json.dumps(job_certificate_cred_def['config']))

    print("\"landlord\" -> Send \"landlord Job-Certificate\" Credential Definition to Ledger")
    await send_cred_def(landlord['pool'], landlord['wallet'], landlord['did'], landlord['job_certificate_cred_def'])

    print("\n=====================================================================")
    print("=== Getting Transcript with bank ==")
    print("\n=====================================================================")
    print("== Tenant setup ==")
    print("\n=====================================================================")

    tenant = {
        'name': 'Tenant',
        'wallet_config': json.dumps({'id': 'tenant_wallet'}),
        'wallet_credentials': json.dumps({'key': 'tenant_wallet_key'}),
        'pool': pool_['handle'],
    }
    await create_wallet(tenant)
    (tenant['did'], tenant['key']) = await did.create_and_store_my_did(tenant['wallet'], "{}")

    print("\n=====================================================================")
    print("== Getting Transcript with bank - Getting Transcript Credential ==")


    print("\"bank\" -> Create \"Transcript\" Credential Offer for tenant")
    bank['transcript_cred_offer'] = \
        await anoncreds.issuer_create_credential_offer(bank['wallet'], bank['transcript_cred_def_id'])

    print("\"bank\" -> Send \"Transcript\" Credential Offer to tenant")
    tenant['transcript_cred_offer'] = bank['transcript_cred_offer']
    transcript_cred_offer_object = json.loads(tenant['transcript_cred_offer'])

    tenant['transcript_schema_id'] = transcript_cred_offer_object['schema_id']
    tenant['transcript_cred_def_id'] = transcript_cred_offer_object['cred_def_id']

    print("\"tenant\" -> Create and store \"tenant\" Master Secret in Wallet")
    tenant['master_secret_id'] = await anoncreds.prover_create_master_secret(tenant['wallet'], None)

    print("\"tenant\" -> Get \"bank Transcript\" Credential Definition from Ledger")
    (tenant['bank_transcript_cred_def_id'], tenant['bank_transcript_cred_def']) = \
        await get_cred_def(tenant['pool'], tenant['did'], tenant['transcript_cred_def_id'])

    print("\"tenant\" -> Create \"Transcript\" Credential Request for bank")
    (tenant['transcript_cred_request'], tenant['transcript_cred_request_metadata']) = \
        await anoncreds.prover_create_credential_req(tenant['wallet'], tenant['did'],
                                                     tenant['transcript_cred_offer'], tenant['bank_transcript_cred_def'],
                                                     tenant['master_secret_id'])

    print("\"tenant\" -> Send \"Transcript\" Credential Request to bank")
    bank['transcript_cred_request'] = tenant['transcript_cred_request']

    tenant['transcript_cred_values'] = json.dumps({
        "first_name": {"raw": "Alice", "encoded": "1139481716457488690172217916278103335"},
        "last_name": {"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
        "degree": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
        "status": {"raw": "graduated", "encoded": "2213454313412354"},
        "ssn": {"raw": "123-45-6789", "encoded": "3124141231422543541"},
        "year": {"raw": "2015", "encoded": "2015"},
        "average": {"raw": "5", "encoded": "5"}
    })
    bank['tenant_transcript_cred_values'] = tenant['transcript_cred_values']

    print("\"bank\" -> Create \"Transcript\" Credential for tenant")

    bank['transcript_cred'], _, _ = \
        await anoncreds.issuer_create_credential(bank['wallet'], bank['transcript_cred_offer'],
                                                 bank['transcript_cred_request'],
                                                 bank['tenant_transcript_cred_values'], None, None)

    print("\"bank\" -> Send \"Transcript\" Credential to tenant")
    tenant['transcript_cred'] = bank['transcript_cred']

    print("\"tenant\" -> Store \"Transcript\" Credential from bank")
    _, tenant['transcript_cred_def'] = await get_cred_def(tenant['pool'], tenant['did'],
                                                         tenant['transcript_cred_def_id'])

    await anoncreds.prover_store_credential(tenant['wallet'], None, tenant['transcript_cred_request_metadata'],
                                            tenant['transcript_cred'], tenant['transcript_cred_def'], None)

    print("\n=====================================================================")
    print("== Apply for the job with landlord - Transcript proving ==")


    print("\"landlord\" -> Create \"Job-Application\" Proof Request")
    nonce = await anoncreds.generate_nonce()
    landlord['job_application_proof_request'] = json.dumps({
        'nonce': nonce,
        'name': 'Job-Application',
        'version': '0.1',
        'requested_attributes': {
            'attr1_referent': {
                'name': 'first_name'
            },
            'attr2_referent': {
                'name': 'last_name'
            },
            'attr3_referent': {
                'name': 'degree',
                'restrictions': [{'cred_def_id': bank['transcript_cred_def_id']}]
            },
            'attr4_referent': {
                'name': 'status',
                'restrictions': [{'cred_def_id': bank['transcript_cred_def_id']}]
            },
            'attr5_referent': {
                'name': 'ssn',
                'restrictions': [{'cred_def_id': bank['transcript_cred_def_id']}]
            },
            'attr6_referent': {
                'name': 'phone_number'
            }
        },
        'requested_predicates': {
            'predicate1_referent': {
                'name': 'average',
                'p_type': '>=',
                'p_value': 4,
                'restrictions': [{'cred_def_id': bank['transcript_cred_def_id']}]
            }
        }
    })

    print("\"landlord\" -> Send \"Job-Application\" Proof Request to tenant")
    tenant['job_application_proof_request'] = landlord['job_application_proof_request']

    print("\"tenant\" -> Get credentials for \"Job-Application\" Proof Request")

    search_for_job_application_proof_request = \
        await anoncreds.prover_search_credentials_for_proof_req(tenant['wallet'],
                                                                tenant['job_application_proof_request'], None)

    cred_for_attr1 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr1_referent')
    cred_for_attr2 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr2_referent')
    cred_for_attr3 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr3_referent')
    cred_for_attr4 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr4_referent')
    cred_for_attr5 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr5_referent')
    cred_for_predicate1 = \
        await get_credential_for_referent(search_for_job_application_proof_request, 'predicate1_referent')

    await anoncreds.prover_close_credentials_search_for_proof_req(search_for_job_application_proof_request)

    tenant['creds_for_job_application_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                                cred_for_attr2['referent']: cred_for_attr2,
                                                cred_for_attr3['referent']: cred_for_attr3,
                                                cred_for_attr4['referent']: cred_for_attr4,
                                                cred_for_attr5['referent']: cred_for_attr5,
                                                cred_for_predicate1['referent']: cred_for_predicate1}

    tenant['schemas'], tenant['cred_defs'], tenant['revoc_states'] = \
        await prover_get_entities_from_ledger(tenant['pool'], tenant['did'],
                                              tenant['creds_for_job_application_proof'], tenant['name'])

    print("\"tenant\" -> Create \"Job-Application\" Proof")
    tenant['job_application_requested_creds'] = json.dumps({
        'self_attested_attributes': {
            'attr1_referent': 'Alice',
            'attr2_referent': 'Garcia',
            'attr6_referent': '123-45-6789'
        },
        'requested_attributes': {
            'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True},
            'attr4_referent': {'cred_id': cred_for_attr4['referent'], 'revealed': True},
            'attr5_referent': {'cred_id': cred_for_attr5['referent'], 'revealed': True},
        },
        'requested_predicates': {'predicate1_referent': {'cred_id': cred_for_predicate1['referent']}}
    })

    tenant['job_application_proof'] = \
        await anoncreds.prover_create_proof(tenant['wallet'], tenant['job_application_proof_request'],
                                            tenant['job_application_requested_creds'], tenant['master_secret_id'],
                                            tenant['schemas'], tenant['cred_defs'], tenant['revoc_states'])

    print("\"tenant\" -> Send \"Job-Application\" Proof to landlord")
    landlord['job_application_proof'] = tenant['job_application_proof']
    job_application_proof_object = json.loads(landlord['job_application_proof'])

    landlord['schemas_for_job_application'], landlord['cred_defs_for_job_application'], \
    landlord['revoc_ref_defs_for_job_application'], landlord['revoc_regs_for_job_application'] = \
        await verifier_get_entities_from_ledger(landlord['pool'], landlord['did'],
                                                job_application_proof_object['identifiers'], landlord['name'])

    print("\"landlord\" -> Verify \"Job-Application\" Proof from tenant")
    assert 'Bachelor of Science, Marketing' == \
           job_application_proof_object['requested_proof']['revealed_attrs']['attr3_referent']['raw']
    assert 'graduated' == \
           job_application_proof_object['requested_proof']['revealed_attrs']['attr4_referent']['raw']
    assert '123-45-6789' == \
           job_application_proof_object['requested_proof']['revealed_attrs']['attr5_referent']['raw']

    assert 'Alice' == job_application_proof_object['requested_proof']['self_attested_attrs']['attr1_referent']
    assert 'Garcia' == job_application_proof_object['requested_proof']['self_attested_attrs']['attr2_referent']
    assert '123-45-6789' == job_application_proof_object['requested_proof']['self_attested_attrs']['attr6_referent']

    assert await anoncreds.verifier_verify_proof(landlord['job_application_proof_request'], landlord['job_application_proof'],
                                                 landlord['schemas_for_job_application'],
                                                 landlord['cred_defs_for_job_application'],
                                                 landlord['revoc_ref_defs_for_job_application'],
                                                 landlord['revoc_regs_for_job_application'])

    print("\n=====================================================================")
    print("== Apply for the job with landlord - Getting Job-Certificate Credential ==")


    print("\"landlord\" -> Create \"Job-Certificate\" Credential Offer for tenant")
    landlord['job_certificate_cred_offer'] = \
        await anoncreds.issuer_create_credential_offer(landlord['wallet'], landlord['job_certificate_cred_def_id'])

    print("\"landlord\" -> Send \"Job-Certificate\" Credential Offer to tenant")
    tenant['job_certificate_cred_offer'] = landlord['job_certificate_cred_offer']
    job_certificate_cred_offer_object = json.loads(tenant['job_certificate_cred_offer'])

    print("\"tenant\" -> Get \"landlord Job-Certificate\" Credential Definition from Ledger")
    (tenant['landlord_job_certificate_cred_def_id'], tenant['landlord_job_certificate_cred_def']) = \
        await get_cred_def(tenant['pool'], tenant['did'], job_certificate_cred_offer_object['cred_def_id'])

    print("\"tenant\" -> Create and store in Wallet \"Job-Certificate\" Credential Request for landlord")
    (tenant['job_certificate_cred_request'], tenant['job_certificate_cred_request_metadata']) = \
        await anoncreds.prover_create_credential_req(tenant['wallet'], tenant['did'],
                                                     tenant['job_certificate_cred_offer'],
                                                     tenant['landlord_job_certificate_cred_def'], tenant['master_secret_id'])

    print("\"tenant\" -> Send \"Job-Certificate\" Credential Request to landlord")
    landlord['job_certificate_cred_request'] = tenant['job_certificate_cred_request']

    tenant['job_certificate_cred_values'] = json.dumps({
        "first_name": {"raw": "Alice", "encoded": "245712572474217942457235975012103335"},
        "last_name": {"raw": "Garcia", "encoded": "312643218496194691632153761283356127"},
        "employee_status": {"raw": "Permanent", "encoded": "2143135425425143112321314321"},
        "salary": {"raw": "2400", "encoded": "2400"},
        "experience": {"raw": "10", "encoded": "10"}
    })
    landlord['job_certificate_cred_values'] = tenant['job_certificate_cred_values']

    print("\"landlord\" -> Create \"Job-Certificate\" Credential for tenant")

    landlord['job_certificate_cred'], _, _ = \
        await anoncreds.issuer_create_credential(landlord['wallet'], landlord['job_certificate_cred_offer'],
                                                 landlord['job_certificate_cred_request'],
                                                 landlord['job_certificate_cred_values'], None, None)

    print("\"landlord\" -> Send \"Job-Certificate\" Credential to tenant")
    tenant['job_certificate_cred'] = landlord['job_certificate_cred']

    print("\"tenant\" -> Store \"Job-Certificate\" Credential")
    await anoncreds.prover_store_credential(tenant['wallet'], None, tenant['job_certificate_cred_request_metadata'],
                                            tenant['job_certificate_cred'],
                                            tenant['landlord_job_certificate_cred_def'], None)

    print("\n=====================================================================")
    print("== Apply for the loan with Thrift - Job-Certificate proving")


    print("\"Thrift\" -> Create \"Loan-Application-Basic\" Proof Request")
    nonce = await anoncreds.generate_nonce()
    thrift['apply_loan_proof_request'] = json.dumps({
        'nonce': nonce,
        'name': 'Loan-Application-Basic',
        'version': '0.1',
        'requested_attributes': {
            'attr1_referent': {
                'name': 'employee_status',
                'restrictions': [{'cred_def_id': landlord['job_certificate_cred_def_id']}]
            }
        },
        'requested_predicates': {
            'predicate1_referent': {
                'name': 'salary',
                'p_type': '>=',
                'p_value': 2000,
                'restrictions': [{'cred_def_id': landlord['job_certificate_cred_def_id']}]
            },
            'predicate2_referent': {
                'name': 'experience',
                'p_type': '>=',
                'p_value': 1,
                'restrictions': [{'cred_def_id': landlord['job_certificate_cred_def_id']}]
            }
        }
    })

    print("\"Thrift\" -> Send \"Loan-Application-Basic\" Proof Request to tenant")
    tenant['apply_loan_proof_request'] = thrift['apply_loan_proof_request']

    print("\"tenant\" -> Get credentials for \"Loan-Application-Basic\" Proof Request")

    search_for_apply_loan_proof_request = \
        await anoncreds.prover_search_credentials_for_proof_req(tenant['wallet'],
                                                                tenant['apply_loan_proof_request'], None)

    cred_for_attr1 = await get_credential_for_referent(search_for_apply_loan_proof_request, 'attr1_referent')
    cred_for_predicate1 = await get_credential_for_referent(search_for_apply_loan_proof_request, 'predicate1_referent')
    cred_for_predicate2 = await get_credential_for_referent(search_for_apply_loan_proof_request, 'predicate2_referent')

    await anoncreds.prover_close_credentials_search_for_proof_req(search_for_apply_loan_proof_request)

    tenant['creds_for_apply_loan_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                           cred_for_predicate1['referent']: cred_for_predicate1,
                                           cred_for_predicate2['referent']: cred_for_predicate2}

    tenant['schemas'], tenant['cred_defs'], tenant['revoc_states'] = \
        await prover_get_entities_from_ledger(tenant['pool'], tenant['did'],
                                              tenant['creds_for_apply_loan_proof'],
                                              tenant['name'])

    print("\"tenant\" -> Create \"Loan-Application-Basic\" Proof")
    tenant['apply_loan_requested_creds'] = json.dumps({
        'self_attested_attributes': {},
        'requested_attributes': {
            'attr1_referent': {'cred_id': cred_for_attr1['referent'], 'revealed': True}
        },
        'requested_predicates': {
            'predicate1_referent': {'cred_id': cred_for_predicate1['referent']},
            'predicate2_referent': {'cred_id': cred_for_predicate2['referent']}
        }
    })
    tenant['apply_loan_proof'] = \
        await anoncreds.prover_create_proof(tenant['wallet'], tenant['apply_loan_proof_request'],
                                            tenant['apply_loan_requested_creds'], tenant['master_secret_id'],
                                            tenant['schemas'], tenant['cred_defs'], tenant['revoc_states'])

    print("\"tenant\" -> Send \"Loan-Application-Basic\" Proof to Thrift")
    thrift['apply_loan_proof'] = tenant['apply_loan_proof']
    apply_loan_proof_object = json.loads(thrift['apply_loan_proof'])

    print("\"Thrift\" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger"
          " required for Proof verifying")

    thrift['schemas'], thrift['cred_defs'], thrift['revoc_defs'], thrift['revoc_regs'] = \
        await verifier_get_entities_from_ledger(thrift['pool'], thrift['did'],
                                                apply_loan_proof_object['identifiers'], thrift['name'])

    print("\"Thrift\" -> Verify \"Loan-Application-Basic\" Proof from tenant")
    assert 'Permanent' == \
           apply_loan_proof_object['requested_proof']['revealed_attrs']['attr1_referent']['raw']

    assert await anoncreds.verifier_verify_proof(thrift['apply_loan_proof_request'], thrift['apply_loan_proof'],
                                                 thrift['schemas'], thrift['cred_defs'], thrift['revoc_defs'],
                                                 thrift['revoc_regs'])

    print("\n=====================================================================")
    print("== Apply for the loan with Thrift - Transcript and Job-Certificate proving")


    print("\"Thrift\" -> Create \"Loan-Application-KYC\" Proof Request")
    nonce = await anoncreds.generate_nonce()
    thrift['apply_loan_kyc_proof_request'] = json.dumps({
        'nonce': nonce,
        'name': 'Loan-Application-KYC',
        'version': '0.1',
        'requested_attributes': {
            'attr1_referent': {'name': 'first_name'},
            'attr2_referent': {'name': 'last_name'},
            'attr3_referent': {'name': 'ssn'}
        },
        'requested_predicates': {}
    })

    print("\"Thrift\" -> Send \"Loan-Application-KYC\" Proof Request to tenant")
    tenant['apply_loan_kyc_proof_request'] = thrift['apply_loan_kyc_proof_request']

    print("\"tenant\" -> Get credentials for \"Loan-Application-KYC\" Proof Request")

    search_for_apply_loan_kyc_proof_request = \
        await anoncreds.prover_search_credentials_for_proof_req(tenant['wallet'],
                                                                tenant['apply_loan_kyc_proof_request'], None)

    cred_for_attr1 = await get_credential_for_referent(search_for_apply_loan_kyc_proof_request, 'attr1_referent')
    cred_for_attr2 = await get_credential_for_referent(search_for_apply_loan_kyc_proof_request, 'attr2_referent')
    cred_for_attr3 = await get_credential_for_referent(search_for_apply_loan_kyc_proof_request, 'attr3_referent')

    await anoncreds.prover_close_credentials_search_for_proof_req(search_for_apply_loan_kyc_proof_request)

    tenant['creds_for_apply_loan_kyc_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                               cred_for_attr2['referent']: cred_for_attr2,
                                               cred_for_attr3['referent']: cred_for_attr3}

    tenant['schemas'], tenant['cred_defs'], tenant['revoc_states'] = \
        await prover_get_entities_from_ledger(tenant['pool'], tenant['did'],
                                              tenant['creds_for_apply_loan_kyc_proof'], 'Tenant')

    print("\"tenant\" -> Create \"Loan-Application-KYC\" Proof")

    tenant['apply_loan_kyc_requested_creds'] = json.dumps({
        'self_attested_attributes': {},
        'requested_attributes': {
            'attr1_referent': {'cred_id': cred_for_attr1['referent'], 'revealed': True},
            'attr2_referent': {'cred_id': cred_for_attr2['referent'], 'revealed': True},
            'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True}
        },
        'requested_predicates': {}
    })

    tenant['apply_loan_kyc_proof'] = \
        await anoncreds.prover_create_proof(tenant['wallet'], tenant['apply_loan_kyc_proof_request'],
                                            tenant['apply_loan_kyc_requested_creds'], tenant['master_secret_id'],
                                            tenant['schemas'], tenant['cred_defs'], tenant['revoc_states'])

    print("\"tenant\" -> Send \"Loan-Application-KYC\" Proof to Thrift")
    thrift['apply_loan_kyc_proof'] = tenant['apply_loan_kyc_proof']
    apply_loan_kyc_proof_object = json.loads(thrift['apply_loan_kyc_proof'])

    print("\"Thrift\" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger"
          " required for Proof verifying")

    thrift['schemas'], thrift['cred_defs'], thrift['revoc_defs'], thrift['revoc_regs'] = \
        await verifier_get_entities_from_ledger(thrift['pool'], thrift['did'],
                                                apply_loan_kyc_proof_object['identifiers'], 'Thrift')

    print("\"Thrift\" -> Verify \"Loan-Application-KYC\" Proof from tenant")
    assert 'Alice' == \
           apply_loan_kyc_proof_object['requested_proof']['revealed_attrs']['attr1_referent']['raw']
    assert 'Garcia' == \
           apply_loan_kyc_proof_object['requested_proof']['revealed_attrs']['attr2_referent']['raw']
    assert '123-45-6789' == \
           apply_loan_kyc_proof_object['requested_proof']['revealed_attrs']['attr3_referent']['raw']

    assert await anoncreds.verifier_verify_proof(thrift['apply_loan_kyc_proof_request'],
                                                 thrift['apply_loan_kyc_proof'],
                                                 thrift['schemas'], thrift['cred_defs'], thrift['revoc_defs'],
                                                 thrift['revoc_regs'])

    print("\n=====================================================================")

    print("\"Sovrin Steward\" -> Close and Delete wallet")
    await wallet.close_wallet(steward['wallet'])
    await wallet.delete_wallet(steward['wallet_config'], steward['wallet_credentials'])

    print("\"rental\" -> Close and Delete wallet")
    await wallet.close_wallet(rental['wallet'])
    await wallet.delete_wallet(rental['wallet_config'], rental['wallet_credentials'])

    print("\"bank\" -> Close and Delete wallet")
    await wallet.close_wallet(bank['wallet'])
    await wallet.delete_wallet(bank['wallet_config'], bank['wallet_credentials'])

    print("\"landlord\" -> Close and Delete wallet")
    await wallet.close_wallet(landlord['wallet'])
    await wallet.delete_wallet(landlord['wallet_config'], landlord['wallet_credentials'])

    print("\"Thrift\" -> Close and Delete wallet")
    await wallet.close_wallet(thrift['wallet'])
    await wallet.delete_wallet(thrift['wallet_config'], thrift['wallet_credentials'])

    print("\"tenant\" -> Close and Delete wallet")
    await wallet.close_wallet(tenant['wallet'])
    await wallet.delete_wallet(tenant['wallet_config'], tenant['wallet_credentials'])

    print("Close and Delete pool")
    await pool.close_pool_ledger(pool_['handle'])
    await pool.delete_pool_ledger_config(pool_['name'])

    print("Getting started -> done")

async def create_wallet(identity):
    print("\"{}\" -> Create wallet".format(identity['name']))
    try:
        await wallet.create_wallet(identity['wallet_config'], identity['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    identity['wallet'] = await wallet.open_wallet(identity['wallet_config'], identity['wallet_credentials'])


async def getting_verinym(from_, to):
    await create_wallet(to)

    (to['did'], to['key']) = await did.create_and_store_my_did(to['wallet'], "{}")

    from_['info'] = {
        'did': to['did'],
        'verkey': to['key'],
        'role': to['role'] or None
    }

    await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
                   from_['info']['verkey'], from_['info']['role'])


async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)


async def send_schema(pool_handle, wallet_handle, _did, schema):
    schema_request = await ledger.build_schema_request(_did, schema)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, schema_request)


async def send_cred_def(pool_handle, wallet_handle, _did, cred_def_json):
    cred_def_request = await ledger.build_cred_def_request(_did, cred_def_json)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, cred_def_request)


async def get_schema(pool_handle, _did, schema_id):
    get_schema_request = await ledger.build_get_schema_request(_did, schema_id)
    get_schema_response = await ledger.submit_request(pool_handle, get_schema_request)
    return await ledger.parse_get_schema_response(get_schema_response)


async def get_cred_def(pool_handle, _did, cred_def_id):
    get_cred_def_request = await ledger.build_get_cred_def_request(_did, cred_def_id)
    get_cred_def_response = await ledger.submit_request(pool_handle, get_cred_def_request)
    return await ledger.parse_get_cred_def_response(get_cred_def_response)


async def get_credential_for_referent(search_handle, referent):
    credentials = json.loads(
        await anoncreds.prover_fetch_credentials_for_proof_req(search_handle, referent, 10))
    return credentials[0]['cred_info']


async def prover_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_states = {}
    for item in identifiers.values():
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Create Revocation States

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_states)


async def verifier_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_reg_defs = {}
    rev_regs = {}
    for item in identifiers:
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Get Revocation Definitions and Revocation Registries

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_reg_defs), json.dumps(rev_regs)

await run()
